# Week 02 — Round‑Robin & Preemption

Dieses Notebook demonstriert den Round-Robin-Scheduler und fürht einen Vergleich mit FIFO durch.

**Ziele**

- Eine einfache FIFO-Strategie verstehen und testen
- Ausprobieren der Referenz-Implementation


**Einstieg**

Führen Sie die unten stehende Zelle, um die Leistungskenngrößen der Scheduling-Strategien FIFO und RR miteinander zu vergleichen.  

In [ ]:
from common.process import Process
from week02.reference.scheduler import FifoScheduler, RoundRobinScheduler
from week02.sim.simulator import run_simulation_fifo, run_simulation_rr

p1 = Process(pid=1, arrival=0.0, burst=20.0)
p2 = Process(pid=2, arrival=0.1, burst=3.0)
p3 = Process(pid=3, arrival=0.2, burst=7.0)
procs = [p1,p2,p3]

fifo = FifoScheduler()
rr = RoundRobinScheduler(quantum=1.0)

res_fifo = run_simulation_fifo(fifo, procs)
res_rr = run_simulation_rr(rr, procs)

print('FIFO results (PID, waiting, turnaround):')
for r in res_fifo:
    print(r)
print('\nRR results (PID, waiting, turnaround, response):')
for r in res_rr:
    print(r)

avg_wait_fifo = sum(w for (_,w,_) in res_fifo)/len(res_fifo)
avg_wait_rr = sum(w for (_,w,_,_) in res_rr)/len(res_rr)

avg_turn_fifo = sum(t for (_,_,t) in res_fifo)/len(res_fifo)
avg_turn_rr = sum(t for (_,_,t,_) in res_rr)/len(res_rr)

avg_resp_rr = sum(r for (_,_,_,r) in res_rr)/len(res_rr)


print('\navg_wait_fifo = ' + "{:.2f}".format(avg_wait_fifo) + "; " + 'avg_wait_rr = ' + "{:.2f}".format(avg_wait_rr))
print('\navg_turn_fifo = ' + "{:.2f}".format(avg_turn_fifo) + "; " + 'avg_turn_rr = ' + "{:.2f}".format(avg_turn_rr))
print('\navg_response_rr = ' + "{:.2f}".format(avg_resp_rr))


### Visualisierung der Ergebnisse

In der unten stehenden Zelle werden die Ergebnisse anhand eines Balken-Diagramms (per PID) mit Waiting / Turnaround visualisiert.

- Führen Sie die Simulation oben mit verschiedenen Prozesseigenschaften
- Vergleichen Sie dabei die Ergebnisse für FIFO und RR


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Benutze dicts (robust gegenüber Reihenfolge) ---
fifo_by_pid = {pid: (wait, turn) for pid, wait, turn in res_fifo}
rr_by_pid   = {pid: (wait, turn) for pid, wait, turn in res_rr}

# Eine konsistente Reihenfolge bei den PIDs verwenden.
# Erste Option: PID-Reihenfolge in res_fifo, Zweite Option: sorted union (else).
if res_fifo:
    pid_order = [pid for pid, _, _ in res_fifo]
else:
    pid_order = sorted(set(list(fifo_by_pid.keys()) + list(rr_by_pid.keys())))

# Wir müssen sicher gehen, dass alle PIDs berücksichtigt werden (Unterschiede zwischen FIFO und RR?)
all_pids = sorted(set(pid_order) | set(rr_by_pid.keys()))
# Die Reiehnfolge in res_fifo behalten und die restlichen hinzufügen
ordered_pids = [p for p in pid_order if p in all_pids] + [p for p in all_pids if p not in pid_order]

# --- Extrahiere ausgerichtete Arrays (falls Werte fehlen, ersetze durch np.nan "Not a Number") ---
def get_vals(pid_list, lookup):
    waits = []
    turns = []
    for pid in pid_list:
        if pid in lookup:
            w, t = lookup[pid]
            waits.append(w)
            turns.append(t)
        else:
            waits.append(np.nan)
            turns.append(np.nan)
    return np.array(waits), np.array(turns)

waits_fifo, turns_fifo = get_vals(ordered_pids, fifo_by_pid)
waits_rr,   turns_rr   = get_vals(ordered_pids, rr_by_pid)

# Mittelwerte berechnen (ignoriere dabei NaNs)
mean_wait_fifo = np.nanmean(waits_fifo)
mean_turn_fifo = np.nanmean(turns_fifo)
mean_wait_rr   = np.nanmean(waits_rr)
mean_turn_rr   = np.nanmean(turns_rr)

# --- Darstellung ---
x = np.arange(len(ordered_pids))
width = 0.35

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5), sharey=False)

# Vergelich: Waiting time
ax1.bar(x - width/2, waits_fifo, width, label='FIFO', alpha=0.8)
ax1.bar(x + width/2, waits_rr, width, label='RR', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels([f'P{pid}' for pid in ordered_pids])
ax1.set_xlabel('PID')
ax1.set_ylabel('Waiting time (s)')
ax1.set_title('Waiting time: FIFO vs RR')
ax1.legend()
ax1.grid(axis='y', linestyle=':', alpha=0.5)

# Vergleich: Turnaround time
ax2.bar(x - width/2, turns_fifo, width, label='FIFO', alpha=0.8)
ax2.bar(x + width/2, turns_rr, width, label='RR', alpha=0.8)
ax2.set_xticks(x)
ax2.set_xticklabels([f'P{pid}' for pid in ordered_pids])
ax2.set_xlabel('PID')
ax2.set_ylabel('Turnaround time (s)')
ax2.set_title('Turnaround time: FIFO vs RR')
ax2.legend()
ax2.grid(axis='y', linestyle=':', alpha=0.5)

# Beschriften
txt = (f'avg_wait FIFO: {mean_wait_fifo:.2f} s\n'
       f'avg_turn FIFO: {mean_turn_fifo:.2f} s\n\n'
       f'avg_wait RR: {mean_wait_rr:.2f} s\n'
       f'avg_turn RR: {mean_turn_rr:.2f} s')
fig.suptitle('Vergleich FIFO vs Round-Robin (pro Prozess + Mittelwerte)', fontsize=14)
plt.subplots_adjust(top=0.85, wspace=0.35)
fig.text(0.5, 0.01, txt, ha='center', va='top', fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.show()

# --- Zusammenfassende Tabelle in der Konsole ausgeben ---
print("PID | wait_FIFO | wait_RR | turn_FIFO | turn_RR")
for i, pid in enumerate(ordered_pids):
    print(f"{pid:3} | {waits_fifo[i]:9.3f} | {waits_rr[i]:7.3f} | {turns_fifo[i]:9.3f} | {turns_rr[i]:7.3f}")
print()
print(f"avg_wait FIFO = {mean_wait_fifo:.3f}, avg_turn FIFO = {mean_turn_fifo:.3f}")
print(f"avg_wait RR   = {mean_wait_rr:.3f}, avg_turn RR   = {mean_turn_rr:.3f}")

### Ein vollständig instrumentierter Simulator

Um ein Gantt-Diagramm mit dem Zeitverlauf der Prozesse erstellen zu können, brauchen wir einen Simulator, der Start und Ende jedes Quantum für jeden Prozess liefert. Einen solchen Simulator finden Sie in in `src/week02/sim` als `run_simulation_rr_instrumented(scheduler, processes: List[Process])`.

Der Code dieses Simulators ist nicht vollständig. Wenn Sie in der unten stehenden Zelle die Simulation durchführen, werden Sie einen Fehler bekommen `NotImplementedError("Student: Ergänzt hier arrivals/reinsertion/context-switch handling.")`. Sie müssen den Code vervollständigen.

Ergänzen Sie den Code, der nach dem Ausführen eines Quants passieren muss. Konkrete Anforderungen:

1. Fügen Sie alle Prozesse ein, die *während* [start, end] angekommen sind, in die ready-Queue (in Ankunftsreihenfolge).
    - *Tipp:* push_arrivals_up_to(time) — benutzen, aber auf die Reihenfolge achten!

2. Falls `p` noch nicht fertig ist (`p.remaining > float_eps`), dann muss `p` ans Ende von ready angefügt werden.
    - **WICHTIG**: Die Neuankömmlinge, die während des Quants eingetroffen sind, müssen *vor* `p` kommen.

3. Wenn das Quantum abgelaufen ist (d.h. `exec_time == quantum`) UND der Prozess nicht fertig ist, so muss ein Context Switch modelliert werden:
`if context_switch_time > 0.0: time += context_switch_time`
    - (Optional: falls `record_switch_segments` `True` ist, zeichnet den CS-Segment).
    - *Hinweis:* Auch wenn danach nur `p` im ready steht, soll trotzdem ein CS stattfinden.

4. Wenn der Prozess fertig ist (`p.remaining <= float_eps`), dann:
    - setze `per_process[p.pid]['finish'] = time`
    - berechne turnaround/waiting/response und füge in completed ein

Tipp: Überlegt Euch die Reihenfolge:

1. arrival handling,
2. reinsertion oder removal von p,
3. context switch time addition

In [ ]:
# Simulation ausführen mit run_simulation_rr_instrumented

from common.process import Process
from week02.reference.scheduler import FifoScheduler, RoundRobinScheduler
from week02.sim.simulator import run_simulation_rr_instrumented

p1 = Process(pid=1, arrival=0.0, burst=20.0)
p2 = Process(pid=2, arrival=0.1, burst=3.0)
p3 = Process(pid=3, arrival=0.2, burst=7.0)
procs = [p1,p2,p3]

# Wir erstellen den RR-Scheduler
rr = RoundRobinScheduler(quantum=1.0)

# Die Simulation ausführen
# Die Ergebnisse sind jetzt ein dict
# context_switch_time = 0.5
res_rr_complete = run_simulation_rr_instrumented(rr, procs, context_switch_time = 0.5, record_switch_segments=True)

# Ausgabe der wichtigsten Kennzahlen
for pid, info in sorted(res_rr_complete['per_process'].items()):
    arrival = info.get('arrival')
    burst = info.get('burst')
    first = info.get('first_start')
    finish = info.get('finish')
    total_run = info.get('total_run')
    print("PID: " + str(pid) + "; Arrival: " + str(arrival) + "; Burst: " + str(burst) + "; First start: " + str(first) + "; Finish: " + str(finish))
print("")

for pid, wait, turn, resp in res_rr_complete['results']:
    print(f"PID {pid}: waiting = {wait:.3f}, turnaround = {turn:.3f}, response = {resp:.3f}")
print("Context switches:", res_rr_complete['context_switches'])
print("")

print("timeline (first 10):", res_rr_complete['timeline'][:10])

### Visualisierung der Ergebnisse als Gantt-Diragramm

Mit der instrumentierten Version des Simulators können wir den zeitlichen Verlauf der Prozessausführung darstellen.

In [ ]:
import matplotlib.pyplot as plt

# Wir gehen davon aus, dass die Ergebnisse in res_rr_complete sind
timeline = res_rr_complete['timeline']
pids = sorted({seg[0] for seg in timeline if seg[0] != 'cs'})
ypos = {pid: i for i, pid in enumerate(pids)}

fig, ax = plt.subplots(figsize=(10, 1 + len(pids)*0.6))
cmap = plt.cm.tab10
for seg in timeline:
    if seg[0] == 'cs':
        # context switch segment
        _, s, e = seg
        ax.broken_barh([(s, e-s)], ( -0.4, 0.8), facecolors='lightgray')
    else:
        pid, s, e = seg
        ax.broken_barh([(s, e-s)], (ypos[pid]-0.4, 0.8), facecolors=[cmap(pid % 10)], edgecolor='black')

ax.set_yticks(list(ypos.values()))
ax.set_yticklabels([f'P{pid}' for pid in pids])
ax.set_xlabel('time (s)')
ax.set_title('Gantt: RR (instrumented)')
plt.tight_layout()
plt.show()